In [1]:
suppressPackageStartupMessages(library(readxl))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(patchwork))
suppressPackageStartupMessages(library(pheatmap))
suppressPackageStartupMessages(library(stringr))
library(hise)
library(plyr)
library(purrr)
library(vegan)
#library(arrow)
library(parallel)
library(ComplexHeatmap)
library(ggsci)
library(tidyverse)

------------------------------------------------------------------------------

You have loaded plyr after dplyr - this is likely to cause problems.
If you need functions from both plyr and dplyr, please load plyr first, then dplyr:
library(plyr); library(dplyr)

------------------------------------------------------------------------------


Attaching package: ‘plyr’


The following objects are masked from ‘package:dplyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize



Attaching package: ‘purrr’


The following object is masked from ‘package:plyr’:

    compact


Loading required package: permute

Loading required package: lattice

This is vegan 2.6-4

Loading required package: grid

ComplexHeatmap version 2.16.0
Bioconductor page: http://bioconductor.org/packages/ComplexHeatmap/
Github page: https://github.com/jokergoo/ComplexHeatmap
Documentation: http://jokergoo.github.io/ComplexHeatmap-reference

If you use it in published research, please cit

# Creating merged df with metadata and pseudobulk mean expression

In [2]:
mean_df<-read.csv("/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/Figure_Notebooks/Intermediate_files/Mean_gene_expression/IL1B_CD14_mono_pseudobulk_mean_normalized_unscaled.csv", row.names=1)

In [3]:
meta_data=read.csv('/home/jupyter/IH-A-Aging-Analysis-Notebooks/Qiuyu-Notebooks/scRNA/hise_meta_data_2024-01-23_fixed.csv')
colnames(meta_data)
#unique(meta_data$pbmc_sample_id)
#length(unique(meta_data$pbmc_sample_id))

[1] "X.1"                                   
 [2] "X"                                     
 [3] "lastUpdated"                           
 [4] "sample.id"                             
 [5] "sample.bridgingControl"                
 [6] "sample.sampleKitGuid"                  
 [7] "sample.visitName"                      
 [8] "sample.visitDetails"                   
 [9] "sample.drawDate"                       
[10] "sample.daysSinceFirstVisit"            
[11] "file.id"                               
[12] "file.name"                             
[13] "file.batchID"                          
[14] "file.panel"                            
[15] "file.pool"                             
[16] "file.fileType"                         
[17] "file.majorVersion"                     
[18] "subject.id"                            
[19] "subject.biologicalSex"                 
[20] "subject.birthYear"                     
[21] "CMV.IgG.Serology.Result.Interpretation"
[22] "subject.ethnicity"                     
[23] "subject.partnerCode"                   
[24] "subject.race"                          
[25] "subject.subjectGuid"                   
[26] "cohort.cohortGuid"                     
[27] "sample.diseaseStatesRecordedAtVisit"   
[28] "pbmc_sample_id"                        
[29] "Sample_ID"                             
[30] "Sample_Draw_date"                      
[31] "Covid_Dose_1"                          
[32] "Covid_Dose_2"                          
[33] "Covid_exlcusion"                       
[34] "BMI"                                   
[35] "file.name.downloaded"                  
[36] "file.path"

In [4]:
dim(mean_df)

[1] 33538    92

In [5]:
head(mean_df, n=2)

,IL1B..CD14.monocyte.PB00001.01_Female_BR1,IL1B..CD14.monocyte.PB00002.01_Male_BR1,IL1B..CD14.monocyte.PB00003.01_Female_BR1,IL1B..CD14.monocyte.PB00004.01_Male_BR1,IL1B..CD14.monocyte.PB00006.01_Female_BR1,IL1B..CD14.monocyte.PB00010.02_Male_BR1,IL1B..CD14.monocyte.PB00012.01_Female_BR1,IL1B..CD14.monocyte.PB00014.01_Male_BR1,IL1B..CD14.monocyte.PB00015.01_Male_BR2,IL1B..CD14.monocyte.PB00016.01_Female_BR2,⋯,IL1B..CD14.monocyte.PB00592.01_Male_BR1,IL1B..CD14.monocyte.PB00599.01_Female_BR2,IL1B..CD14.monocyte.PB00600.01_Male_BR2,IL1B..CD14.monocyte.PB00602.01_Female_BR2,IL1B..CD14.monocyte.PB00621.01_Female_BR1,IL1B..CD14.monocyte.PB00623.01_Male_BR2,IL1B..CD14.monocyte.PB00625.02_Male_BR2,IL1B..CD14.monocyte.PB00645.01_Female_BR1,IL1B..CD14.monocyte.PB01425.01_Female_BR1,IL1B..CD14.monocyte.PB01446.01_Male_BR1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
MIR1302-2HG,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
FAM138A,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [6]:
meta<-data.frame(colnames(mean_df))
head(meta, n = 3)

,colnames.mean_df.
,<chr>
1,IL1B..CD14.monocyte.PB00001.01_Female_BR1
2,IL1B..CD14.monocyte.PB00002.01_Male_BR1
3,IL1B..CD14.monocyte.PB00003.01_Female_BR1


In [7]:
names(meta)[names(meta) == "colnames.mean_df."] <- "ID"
head(meta)

length(unique(meta$colnames.mean_df.))

,ID
,<chr>
1,IL1B..CD14.monocyte.PB00001.01_Female_BR1
2,IL1B..CD14.monocyte.PB00002.01_Male_BR1
3,IL1B..CD14.monocyte.PB00003.01_Female_BR1
4,IL1B..CD14.monocyte.PB00004.01_Male_BR1
5,IL1B..CD14.monocyte.PB00006.01_Female_BR1
6,IL1B..CD14.monocyte.PB00010.02_Male_BR1


[1] 0

In [27]:
# # Sample string
# input_string <- "IL1B..CD14.monocyte.PB03924.002_Female_BR1"

# # Extract the desired string using regex
# extracted_string <- sub(".*\\.([^.]+)\\.([^.]+)_.*", "\\1-\\2", input_string)

# print(extracted_string)


In [28]:

# # Extract PB.... ids from the 'ID' column
# meta$pbmc_sample_id <- gsub("IL1B..CD14.monocyte.(PB[0-9]+\\.\\d+)_.*", "\\1", meta$ID)
# meta$pbmc_sample_id  = df['PB_ID'].str.replace('.', '-')
# # Display the extracted ids
# head(meta)

# unique(meta$pbmc_sample_id)

In [8]:
# Sample strings
#input_strings <- c("IL1B..CD14.monocyte.PB02554.002_Female_BR1", "IL1B..CD14.monocyte.PB02554.02_Male_BR1")

# Extract the desired strings using regex
meta$pbmc_sample_id  <- sub(".*\\.([^.]+)\\.([^.]+(?:\\.\\d+)?)_.*", "\\1-\\2", meta$ID)
meta$pbmc_sample_id  <- gsub('_.*','',meta$pbmc_sample_id )

head(meta)
length(unique(meta$pbmc_sample_id))

,ID,pbmc_sample_id
,<chr>,<chr>
1,IL1B..CD14.monocyte.PB00001.01_Female_BR1,PB00001-01
2,IL1B..CD14.monocyte.PB00002.01_Male_BR1,PB00002-01
3,IL1B..CD14.monocyte.PB00003.01_Female_BR1,PB00003-01
4,IL1B..CD14.monocyte.PB00004.01_Male_BR1,PB00004-01
5,IL1B..CD14.monocyte.PB00006.01_Female_BR1,PB00006-01
6,IL1B..CD14.monocyte.PB00010.02_Male_BR1,PB00010-02


[1] 92

In [30]:
# ext_strings <- gsub('_.*','',extracted_strings)
# ext_strings

In [31]:
# meta$pbmc_sample_id <- gsub("\\.", "-", regmatches(meta$ID, regexpr("PB\\d+\\.\\d+", meta$ID)))
# head(meta)

In [9]:
# Left join the dataframes by the common column
merged_df <- left_join(meta, meta_data, by = "pbmc_sample_id")
head(merged_df)

,ID,pbmc_sample_id,X.1,X,lastUpdated,sample.id,sample.bridgingControl,sample.sampleKitGuid,sample.visitName,sample.visitDetails,⋯,cohort.cohortGuid,sample.diseaseStatesRecordedAtVisit,Sample_ID,Sample_Draw_date,Covid_Dose_1,Covid_Dose_2,Covid_exlcusion,BMI,file.name.downloaded,file.path
,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<lgl>,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,IL1B..CD14.monocyte.PB00001.01_Female_BR1,PB00001-01,1,1,2023-11-18T02:03:08.347Z,f499ff83-e513-4d24-a10f-151348269fff,false,KT00001,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,⋯,BR1,NA,NA,2019-10-15,NA,NA,no,<25-BMI,B001-P1_PB00001-01_labeled.h5,/home/jupyter/BRI_Analysis/scRNA/cache/fec489f9-9a74-4635-aa91-d2bf09d1faec/B001-P1_PB00001-01_labeled.h5
2,IL1B..CD14.monocyte.PB00002.01_Male_BR1,PB00002-01,2,2,2023-11-18T02:03:08.347Z,750e90a9-a296-4f0f-969f-60225c2bca17,false,KT00002,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,⋯,BR1,NA,NA,2019-10-15,2020-12-28,2021-01-18,no,<25-BMI,B001-P1_PB00002-01_labeled.h5,/home/jupyter/BRI_Analysis/scRNA/cache/7c0c7979-eebd-4aba-b5b2-6e76b4643623/B001-P1_PB00002-01_labeled.h5
3,IL1B..CD14.monocyte.PB00003.01_Female_BR1,PB00003-01,3,3,2023-11-18T02:03:08.347Z,2db6fb3f-e3f4-454b-891b-9b068541b51d,false,KT00003,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,⋯,BR1,NA,NA,2019-10-15,2020-12-28,2021-01-16,no,<25-BMI,B001-P1_PB00003-01_labeled.h5,/home/jupyter/BRI_Analysis/scRNA/cache/40efd03a-cb2f-4677-af42-a056cbfe5a17/B001-P1_PB00003-01_labeled.h5
4,IL1B..CD14.monocyte.PB00004.01_Male_BR1,PB00004-01,4,4,2023-11-19T10:34:34.12Z,f04693c5-563c-4b5b-ae58-877d0d9ae2fe,false,KT00004,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,⋯,BR1,NA,NA,2019-10-21,2021-04-16,2021-05-06,no,<25-BMI,B002-P1_PB00004-01_2023-11-17T21:36:51.794326181Z_labeled.h5,/home/jupyter/BRI_Analysis/scRNA/cache/68fbcd34-1d63-461d-8195-df5b8dc61b31/B002-P1_PB00004-01_2023-11-17T21:36:51.794326181Z_labeled.h5
5,IL1B..CD14.monocyte.PB00006.01_Female_BR1,PB00006-01,5,5,2023-11-19T05:07:36.805Z,eb5b3a3d-002e-40a6-aa19-aa0e6a7fff8f,false,KT00006,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,⋯,BR1,NA,NA,2019-10-23,2021-01-16,2021-02-26,no,<25-BMI,B002-P1_PB00006-01_2023-11-17T21:36:51.794326181Z_labeled.h5,/home/jupyter/BRI_Analysis/scRNA/cache/ea8d98e9-e99e-4dc6-9e78-9866e0deac68/B002-P1_PB00006-01_2023-11-17T21:36:51.794326181Z_labeled.h5
6,IL1B..CD14.monocyte.PB00010.02_Male_BR1,PB00010-02,6,6,2023-11-18T02:03:08.347Z,f32ac061-0ede-4040-8d8c-35229ea5229c,false,KT00010,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,⋯,BR1,NA,NA,2019-10-28,2021-03-21,2021-04-18,no,>25-30BMI,B078-P2_PB00010-02_2021-08-19T17:09:29.934849811Z_labeled.h5,/home/jupyter/BRI_Analysis/scRNA/cache/237e41df-5616-4852-8a8f-b32835c9efcb/B078-P2_PB00010-02_2021-08-19T17:09:29.934849811Z_labeled.h5


In [10]:
rownames(merged_df)<-merged_df$ID
head(merged_df)

colnames(merged_df)
length(merged_df$pbmc_sample_id)

,ID,pbmc_sample_id,X.1,X,lastUpdated,sample.id,sample.bridgingControl,sample.sampleKitGuid,sample.visitName,sample.visitDetails,⋯,cohort.cohortGuid,sample.diseaseStatesRecordedAtVisit,Sample_ID,Sample_Draw_date,Covid_Dose_1,Covid_Dose_2,Covid_exlcusion,BMI,file.name.downloaded,file.path
,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<lgl>,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
IL1B..CD14.monocyte.PB00001.01_Female_BR1,IL1B..CD14.monocyte.PB00001.01_Female_BR1,PB00001-01,1,1,2023-11-18T02:03:08.347Z,f499ff83-e513-4d24-a10f-151348269fff,false,KT00001,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,⋯,BR1,NA,NA,2019-10-15,NA,NA,no,<25-BMI,B001-P1_PB00001-01_labeled.h5,/home/jupyter/BRI_Analysis/scRNA/cache/fec489f9-9a74-4635-aa91-d2bf09d1faec/B001-P1_PB00001-01_labeled.h5
IL1B..CD14.monocyte.PB00002.01_Male_BR1,IL1B..CD14.monocyte.PB00002.01_Male_BR1,PB00002-01,2,2,2023-11-18T02:03:08.347Z,750e90a9-a296-4f0f-969f-60225c2bca17,false,KT00002,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,⋯,BR1,NA,NA,2019-10-15,2020-12-28,2021-01-18,no,<25-BMI,B001-P1_PB00002-01_labeled.h5,/home/jupyter/BRI_Analysis/scRNA/cache/7c0c7979-eebd-4aba-b5b2-6e76b4643623/B001-P1_PB00002-01_labeled.h5
IL1B..CD14.monocyte.PB00003.01_Female_BR1,IL1B..CD14.monocyte.PB00003.01_Female_BR1,PB00003-01,3,3,2023-11-18T02:03:08.347Z,2db6fb3f-e3f4-454b-891b-9b068541b51d,false,KT00003,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,⋯,BR1,NA,NA,2019-10-15,2020-12-28,2021-01-16,no,<25-BMI,B001-P1_PB00003-01_labeled.h5,/home/jupyter/BRI_Analysis/scRNA/cache/40efd03a-cb2f-4677-af42-a056cbfe5a17/B001-P1_PB00003-01_labeled.h5
IL1B..CD14.monocyte.PB00004.01_Male_BR1,IL1B..CD14.monocyte.PB00004.01_Male_BR1,PB00004-01,4,4,2023-11-19T10:34:34.12Z,f04693c5-563c-4b5b-ae58-877d0d9ae2fe,false,KT00004,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,⋯,BR1,NA,NA,2019-10-21,2021-04-16,2021-05-06,no,<25-BMI,B002-P1_PB00004-01_2023-11-17T21:36:51.794326181Z_labeled.h5,/home/jupyter/BRI_Analysis/scRNA/cache/68fbcd34-1d63-461d-8195-df5b8dc61b31/B002-P1_PB00004-01_2023-11-17T21:36:51.794326181Z_labeled.h5
IL1B..CD14.monocyte.PB00006.01_Female_BR1,IL1B..CD14.monocyte.PB00006.01_Female_BR1,PB00006-01,5,5,2023-11-19T05:07:36.805Z,eb5b3a3d-002e-40a6-aa19-aa0e6a7fff8f,false,KT00006,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,⋯,BR1,NA,NA,2019-10-23,2021-01-16,2021-02-26,no,<25-BMI,B002-P1_PB00006-01_2023-11-17T21:36:51.794326181Z_labeled.h5,/home/jupyter/BRI_Analysis/scRNA/cache/ea8d98e9-e99e-4dc6-9e78-9866e0deac68/B002-P1_PB00006-01_2023-11-17T21:36:51.794326181Z_labeled.h5
IL1B..CD14.monocyte.PB00010.02_Male_BR1,IL1B..CD14.monocyte.PB00010.02_Male_BR1,PB00010-02,6,6,2023-11-18T02:03:08.347Z,f32ac061-0ede-4040-8d8c-35229ea5229c,false,KT00010,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,⋯,BR1,NA,NA,2019-10-28,2021-03-21,2021-04-18,no,>25-30BMI,B078-P2_PB00010-02_2021-08-19T17:09:29.934849811Z_labeled.h5,/home/jupyter/BRI_Analysis/scRNA/cache/237e41df-5616-4852-8a8f-b32835c9efcb/B078-P2_PB00010-02_2021-08-19T17:09:29.934849811Z_labeled.h5


[1] "ID"                                    
 [2] "pbmc_sample_id"                        
 [3] "X.1"                                   
 [4] "X"                                     
 [5] "lastUpdated"                           
 [6] "sample.id"                             
 [7] "sample.bridgingControl"                
 [8] "sample.sampleKitGuid"                  
 [9] "sample.visitName"                      
[10] "sample.visitDetails"                   
[11] "sample.drawDate"                       
[12] "sample.daysSinceFirstVisit"            
[13] "file.id"                               
[14] "file.name"                             
[15] "file.batchID"                          
[16] "file.panel"                            
[17] "file.pool"                             
[18] "file.fileType"                         
[19] "file.majorVersion"                     
[20] "subject.id"                            
[21] "subject.biologicalSex"                 
[22] "subject.birthYear"                     
[23] "CMV.IgG.Serology.Result.Interpretation"
[24] "subject.ethnicity"                     
[25] "subject.partnerCode"                   
[26] "subject.race"                          
[27] "subject.subjectGuid"                   
[28] "cohort.cohortGuid"                     
[29] "sample.diseaseStatesRecordedAtVisit"   
[30] "Sample_ID"                             
[31] "Sample_Draw_date"                      
[32] "Covid_Dose_1"                          
[33] "Covid_Dose_2"                          
[34] "Covid_exlcusion"                       
[35] "BMI"                                   
[36] "file.name.downloaded"                  
[37] "file.path"

[1] 92

# Subsetting for each age and Sex individually

In [22]:
#subset for each sex and age
merged_df_v2 <- merged_df[merged_df$subject.biologicalSex == "Female" & merged_df$cohort.cohortGuid == "BR2", ]

#create a column called "Gene"
mean_df['Gene']= rownames(mean_df)

#filter the gene in count matrix
df <- mean_df %>% filter (Gene %in% c('G0S2','TNF', 'IL1B', 'NR4A1', 'PTGS2', 'BCL2A1', 'BCL6', 'NR4A1', 'METRNL', 'TNFRSF1B',
                                      'FKBP5', 'RBP7', 'UBE2D1', 'FCGR3A', 'CCL4', 'CCL4L2','NFKBID', 'LGALS2', 'CCR1','RASGEF1B', 
                                      'CXCL8', 'NLRP3', 'KLF6', 'EIF1AX', 'JUND'))
#remove column "gene" 
df <- df[,!(names(df) %in% c('Gene'))]

#transpose the data
df <- df %>% t() %>% as.data.frame()

#create a ID column for merging
df['ID']= rownames(df)

#merge the merged_df_v2 (with metadata) and the modified pseudobulk dataframe together
data <- inner_join(df,merged_df_v2,by=c('ID'))

#save the df
write_csv(data, file = "/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/Figure_Notebooks/Intermediate_files/Mean_gene_expression/CSVs_mean_gene_exp_by_age_sex/IL1B_CD14mono_Female_BR2_mean_gene_exp.csv")

In [23]:
#subset for each sex and age
merged_df_v2 <- merged_df[merged_df$subject.biologicalSex == "Female" & merged_df$cohort.cohortGuid == "BR1", ]

#create a column called "Gene"
mean_df['Gene']= rownames(mean_df)

#filter the gene in count matrix
df <- mean_df %>% filter (Gene %in% c('G0S2','TNF', 'IL1B', 'NR4A1', 'PTGS2', 'BCL2A1', 'BCL6', 'NR4A1', 'METRNL', 'TNFRSF1B',
                                      'FKBP5', 'RBP7', 'UBE2D1', 'FCGR3A', 'CCL4', 'CCL4L2','NFKBID', 'LGALS2', 'CCR1','RASGEF1B', 
                                      'CXCL8', 'NLRP3', 'KLF6', 'EIF1AX', 'JUND'))
#remove column "gene" 
df <- df[,!(names(df) %in% c('Gene'))]

#transpose the data
df <- df %>% t() %>% as.data.frame()

#create a ID column for merging
df['ID']= rownames(df)

#merge the merged_df_v2 (with metadata) and the modified pseudobulk dataframe together
data <- inner_join(df,merged_df_v2,by=c('ID'))

#save the df
write_csv(data, file = "/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/Figure_Notebooks/Intermediate_files/Mean_gene_expression/CSVs_mean_gene_exp_by_age_sex/IL1B_CD14mono_Female_BR1_mean_gene_exp.csv")

In [24]:
#subset for each sex and age
merged_df_v2 <- merged_df[merged_df$subject.biologicalSex == "Male" & merged_df$cohort.cohortGuid == "BR2", ]

#create a column called "Gene"
mean_df['Gene']= rownames(mean_df)

#filter the gene in count matrix
df <- mean_df %>% filter (Gene %in% c('G0S2','TNF', 'IL1B', 'NR4A1', 'PTGS2', 'BCL2A1', 'BCL6', 'NR4A1', 'METRNL', 'TNFRSF1B',
                                      'FKBP5', 'RBP7', 'UBE2D1', 'FCGR3A', 'CCL4', 'CCL4L2','NFKBID', 'LGALS2', 'CCR1','RASGEF1B', 
                                      'CXCL8', 'NLRP3', 'KLF6', 'EIF1AX', 'JUND'))
#remove column "gene" 
df <- df[,!(names(df) %in% c('Gene'))]

#transpose the data
df <- df %>% t() %>% as.data.frame()

#create a ID column for merging
df['ID']= rownames(df)

#merge the merged_df_v2 (with metadata) and the modified pseudobulk dataframe together
data <- inner_join(df,merged_df_v2,by=c('ID'))

#save the df
write_csv(data, file = "/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/Figure_Notebooks/Intermediate_files/Mean_gene_expression/CSVs_mean_gene_exp_by_age_sex/IL1B_CD14mono_Male_BR2_mean_gene_exp.csv")

In [25]:
#subset for each sex and age
merged_df_v2 <- merged_df[merged_df$subject.biologicalSex == "Male" & merged_df$cohort.cohortGuid == "BR1", ]

#create a column called "Gene"
mean_df['Gene']= rownames(mean_df)

#filter the gene in count matrix
df <- mean_df %>% filter (Gene %in% c('G0S2','TNF', 'IL1B', 'NR4A1', 'PTGS2', 'BCL2A1', 'BCL6', 'NR4A1', 'METRNL', 'TNFRSF1B',
                                      'FKBP5', 'RBP7', 'UBE2D1', 'FCGR3A', 'CCL4', 'CCL4L2','NFKBID', 'LGALS2', 'CCR1','RASGEF1B', 
                                      'CXCL8', 'NLRP3', 'KLF6', 'EIF1AX', 'JUND'))
#remove column "gene" 
df <- df[,!(names(df) %in% c('Gene'))]

#transpose the data
df <- df %>% t() %>% as.data.frame()

#create a ID column for merging
df['ID']= rownames(df)

#merge the merged_df_v2 (with metadata) and the modified pseudobulk dataframe together
data <- inner_join(df,merged_df_v2,by=c('ID'))

#save the df
write_csv(data, file = "/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/Figure_Notebooks/Intermediate_files/Mean_gene_expression/CSVs_mean_gene_exp_by_age_sex/IL1B_CD14mono_Male_BR1_mean_gene_exp.csv")

## Old working code

In [11]:
merged_df_v2 <- merged_df[merged_df$subject.biologicalSex == "Female" & merged_df$cohort.cohortGuid == "BR2", ]

In [12]:
unique(merged_df_v2$subject.biologicalSex)
unique(merged_df_v2$cohort.cohortGuid)

[1] "Female"

[1] "BR2"

In [13]:
head(mean_df)
length(colnames(mean_df))
#rownames(mean_df)

,IL1B..CD14.monocyte.PB00001.01_Female_BR1,IL1B..CD14.monocyte.PB00002.01_Male_BR1,IL1B..CD14.monocyte.PB00003.01_Female_BR1,IL1B..CD14.monocyte.PB00004.01_Male_BR1,IL1B..CD14.monocyte.PB00006.01_Female_BR1,IL1B..CD14.monocyte.PB00010.02_Male_BR1,IL1B..CD14.monocyte.PB00012.01_Female_BR1,IL1B..CD14.monocyte.PB00014.01_Male_BR1,IL1B..CD14.monocyte.PB00015.01_Male_BR2,IL1B..CD14.monocyte.PB00016.01_Female_BR2,⋯,IL1B..CD14.monocyte.PB00592.01_Male_BR1,IL1B..CD14.monocyte.PB00599.01_Female_BR2,IL1B..CD14.monocyte.PB00600.01_Male_BR2,IL1B..CD14.monocyte.PB00602.01_Female_BR2,IL1B..CD14.monocyte.PB00621.01_Female_BR1,IL1B..CD14.monocyte.PB00623.01_Male_BR2,IL1B..CD14.monocyte.PB00625.02_Male_BR2,IL1B..CD14.monocyte.PB00645.01_Female_BR1,IL1B..CD14.monocyte.PB01425.01_Female_BR1,IL1B..CD14.monocyte.PB01446.01_Male_BR1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
MIR1302-2HG,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0.00000000,0,0,0.000000000,0,0,0,0
FAM138A,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0.00000000,0,0,0.000000000,0,0,0,0
OR4F5,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0.00000000,0,0,0.000000000,0,0,0,0
AL627309.1,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0.02579722,0,0,0.005713804,0,0,0,0
AL627309.3,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0.00000000,0,0,0.000000000,0,0,0,0
AL627309.2,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0.00000000,0,0,0.000000000,0,0,0,0


[1] 92

In [14]:
mean_df['Gene']= rownames(mean_df)
head(mean_df)

#write.csv(mean_df, "/home/jupyter/Myeloid_cells/files/count_matrix_IL1B_CD14_mono.csv")

,IL1B..CD14.monocyte.PB00001.01_Female_BR1,IL1B..CD14.monocyte.PB00002.01_Male_BR1,IL1B..CD14.monocyte.PB00003.01_Female_BR1,IL1B..CD14.monocyte.PB00004.01_Male_BR1,IL1B..CD14.monocyte.PB00006.01_Female_BR1,IL1B..CD14.monocyte.PB00010.02_Male_BR1,IL1B..CD14.monocyte.PB00012.01_Female_BR1,IL1B..CD14.monocyte.PB00014.01_Male_BR1,IL1B..CD14.monocyte.PB00015.01_Male_BR2,IL1B..CD14.monocyte.PB00016.01_Female_BR2,⋯,IL1B..CD14.monocyte.PB00599.01_Female_BR2,IL1B..CD14.monocyte.PB00600.01_Male_BR2,IL1B..CD14.monocyte.PB00602.01_Female_BR2,IL1B..CD14.monocyte.PB00621.01_Female_BR1,IL1B..CD14.monocyte.PB00623.01_Male_BR2,IL1B..CD14.monocyte.PB00625.02_Male_BR2,IL1B..CD14.monocyte.PB00645.01_Female_BR1,IL1B..CD14.monocyte.PB01425.01_Female_BR1,IL1B..CD14.monocyte.PB01446.01_Male_BR1,Gene
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
MIR1302-2HG,0,0,0,0,0,0,0,0,0,0,⋯,0,0.00000000,0,0,0.000000000,0,0,0,0,MIR1302-2HG
FAM138A,0,0,0,0,0,0,0,0,0,0,⋯,0,0.00000000,0,0,0.000000000,0,0,0,0,FAM138A
OR4F5,0,0,0,0,0,0,0,0,0,0,⋯,0,0.00000000,0,0,0.000000000,0,0,0,0,OR4F5
AL627309.1,0,0,0,0,0,0,0,0,0,0,⋯,0,0.02579722,0,0,0.005713804,0,0,0,0,AL627309.1
AL627309.3,0,0,0,0,0,0,0,0,0,0,⋯,0,0.00000000,0,0,0.000000000,0,0,0,0,AL627309.3
AL627309.2,0,0,0,0,0,0,0,0,0,0,⋯,0,0.00000000,0,0,0.000000000,0,0,0,0,AL627309.2


In [15]:
#filter the gene in count matrix

df <- mean_df %>% filter (Gene %in% c('G0S2','TNF', 'IL1B', 'NR4A1', 'PTGS2', 'BCL2A1', 'BCL6', 'NR4A1', 'METRNL', 'TNFRSF1B',
                                      'FKBP5', 'RBP7', 'UBE2D1', 'FCGR3A', 'CCL4', 'CCL4L2','NFKBID', 'LGALS2', 'CCR1','RASGEF1B', 
                                      'CXCL8', 'NLRP3', 'KLF6', 'EIF1AX', 'JUND'))

In [16]:
#NFKBID, RASGEF1B, BCL6, FKBP5, UBE2D1, 'NLRP3', 'KLF6', 'EIF1AX', 'JUND' <- not found
#FKBP5 <- corticoid inflammation gene
#

In [17]:
df
#length(df$IL1B..CD14.monocyte.PB00001.01_Female_BR1)
unique(df$Gene)

,IL1B..CD14.monocyte.PB00001.01_Female_BR1,IL1B..CD14.monocyte.PB00002.01_Male_BR1,IL1B..CD14.monocyte.PB00003.01_Female_BR1,IL1B..CD14.monocyte.PB00004.01_Male_BR1,IL1B..CD14.monocyte.PB00006.01_Female_BR1,IL1B..CD14.monocyte.PB00010.02_Male_BR1,IL1B..CD14.monocyte.PB00012.01_Female_BR1,IL1B..CD14.monocyte.PB00014.01_Male_BR1,IL1B..CD14.monocyte.PB00015.01_Male_BR2,IL1B..CD14.monocyte.PB00016.01_Female_BR2,⋯,IL1B..CD14.monocyte.PB00599.01_Female_BR2,IL1B..CD14.monocyte.PB00600.01_Male_BR2,IL1B..CD14.monocyte.PB00602.01_Female_BR2,IL1B..CD14.monocyte.PB00621.01_Female_BR1,IL1B..CD14.monocyte.PB00623.01_Male_BR2,IL1B..CD14.monocyte.PB00625.02_Male_BR2,IL1B..CD14.monocyte.PB00645.01_Female_BR1,IL1B..CD14.monocyte.PB01425.01_Female_BR1,IL1B..CD14.monocyte.PB01446.01_Male_BR1,Gene
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
RBP7,0.6580341,0.8267947,1.0074245,1.1302947,0.77915792,1.00647253,1.3323505,1.0350740,0.5896210,0.9786370,⋯,0.7928386,1.06375205,0.8745921,0.7469651,1.0505479,0.0000000,1.0012400,1.37013869,0.7956115,RBP7
TNFRSF1B,1.2646416,1.4997217,0.5726240,1.2851622,1.46080800,1.10197095,1.1686409,1.3712373,1.3730211,1.5813378,⋯,1.2157416,1.40579334,1.2816287,1.3333011,1.2994506,1.3842485,1.5909686,1.06614728,1.4928116,TNFRSF1B
FCGR3A,0.2063345,0.1896476,0.0000000,0.2181170,0.51237472,0.24582000,0.3546107,0.2299043,0.1221534,0.4049980,⋯,0.4667233,0.25003737,0.2149161,0.4477868,0.1793682,0.0000000,0.0000000,0.00000000,0.2199266,FCGR3A
PTGS2,0.4037394,0.3446376,0.1164149,1.1987074,0.34677196,1.24186075,0.4829994,0.2771626,0.5801127,0.5792147,⋯,0.6212935,0.50814002,0.5533753,0.8654874,0.4398782,1.8283400,0.6938911,0.38231979,0.6158994,PTGS2
G0S2,0.3681518,1.2006851,0.0000000,0.5537455,0.06202377,0.68290521,0.5757327,0.4888096,0.2909091,0.8516766,⋯,0.7079901,0.27933399,0.3638164,0.0000000,0.8105501,1.2431300,0.4686418,0.36349363,0.1589934,G0S2
NLRP3,0.2194822,0.2702370,0.5425574,0.8469841,0.40963067,0.44083116,0.7558058,0.3235502,0.5995581,0.5352163,⋯,0.5237548,0.44509940,0.4961413,0.5655765,0.6804465,0.9365257,0.4126855,0.29654733,0.4815437,NLRP3
IL1B,0.6693585,0.8664402,0.6734207,0.9965455,1.55288792,1.46812882,0.8223424,0.6474077,1.4278196,1.0405723,⋯,1.3217057,1.49715970,1.2916855,0.5953855,1.6441042,3.0959512,1.1244188,1.74867104,1.0121349,IL1B
CCR1,0.6832188,0.6309584,0.7315364,0.7241851,0.65655656,1.03271548,0.1646967,0.4112466,0.5779835,0.5309352,⋯,0.6002498,0.43270577,0.4550728,0.4235760,0.5106632,0.0000000,0.2232902,0.42976890,0.7072403,CCR1
BCL6,0.2864497,0.0000000,0.1164149,0.2293325,0.30909220,0.25190232,0.0000000,0.2310151,0.2772842,0.3401682,⋯,0.5080019,0.38466344,0.4732558,0.4159875,0.3119816,0.4820303,0.3280479,0.54754719,0.5206769,BCL6


[1] "RBP7"     "TNFRSF1B" "FCGR3A"   "PTGS2"    "G0S2"     "NLRP3"   
 [7] "IL1B"     "CCR1"     "BCL6"     "CXCL8"    "RASGEF1B" "TNF"     
[13] "FKBP5"    "EIF1AX"   "KLF6"     "UBE2D1"   "NR4A1"    "BCL2A1"  
[19] "CCL4"     "CCL4L2"   "METRNL"   "JUND"     "NFKBID"   "LGALS2"

In [18]:
df <- df[,!(names(df) %in% c('Gene'))]

In [19]:
df <- df %>% t() %>% as.data.frame()

In [20]:
df %>% head()


,RBP7,TNFRSF1B,FCGR3A,PTGS2,G0S2,NLRP3,IL1B,CCR1,BCL6,CXCL8,⋯,KLF6,UBE2D1,NR4A1,BCL2A1,CCL4,CCL4L2,METRNL,JUND,NFKBID,LGALS2
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
IL1B..CD14.monocyte.PB00001.01_Female_BR1,0.6580341,1.264642,0.2063345,0.4037394,0.36815175,0.2194822,0.6693585,0.6832188,0.2864497,0.9804992,⋯,1.3903171,0.7084627,0.1763264,1.0854804,0.2647265,0.2521105,0.3153873,1.802971,0.4359790,0.8965622
IL1B..CD14.monocyte.PB00002.01_Male_BR1,0.8267947,1.499722,0.1896476,0.3446376,1.20068510,0.2702370,0.8664402,0.6309584,0.0000000,0.3246152,⋯,1.3804124,1.0598155,0.0000000,1.0461003,0.3246152,0.0000000,0.3706055,2.202253,0.1771037,1.6173190
IL1B..CD14.monocyte.PB00003.01_Female_BR1,1.0074245,0.572624,0.0000000,0.1164149,0.00000000,0.5425574,0.6734207,0.7315364,0.1164149,0.9467017,⋯,0.5790715,1.4168231,0.2696620,0.7838191,0.0000000,0.0000000,0.1842882,1.958701,0.0000000,0.8423586
IL1B..CD14.monocyte.PB00004.01_Male_BR1,1.1302947,1.285162,0.2181170,1.1987074,0.55374549,0.8469841,0.9965455,0.7241851,0.2293325,1.0225037,⋯,0.9260696,1.6834447,0.2793685,0.4779797,0.1875182,0.1200838,0.4924537,1.592984,0.4773711,1.7701366
IL1B..CD14.monocyte.PB00006.01_Female_BR1,0.7791579,1.460808,0.5123747,0.3467720,0.06202377,0.4096307,1.5528879,0.6565566,0.3090922,0.7221460,⋯,1.4635338,0.8891441,0.2998147,0.9152629,0.9778832,0.1912439,0.5341154,1.697275,0.1670615,0.3257211
IL1B..CD14.monocyte.PB00010.02_Male_BR1,1.0064725,1.101971,0.2458200,1.2418607,0.68290521,0.4408312,1.4681288,1.0327155,0.2519023,0.5140556,⋯,1.3836930,0.8436514,1.1180331,1.6009474,0.7034707,0.6336593,0.5660796,1.570037,0.3828403,1.4431669


In [21]:
df['ID']= rownames(df)
df %>% head()

,RBP7,TNFRSF1B,FCGR3A,PTGS2,G0S2,NLRP3,IL1B,CCR1,BCL6,CXCL8,⋯,UBE2D1,NR4A1,BCL2A1,CCL4,CCL4L2,METRNL,JUND,NFKBID,LGALS2,ID
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
IL1B..CD14.monocyte.PB00001.01_Female_BR1,0.6580341,1.264642,0.2063345,0.4037394,0.36815175,0.2194822,0.6693585,0.6832188,0.2864497,0.9804992,⋯,0.7084627,0.1763264,1.0854804,0.2647265,0.2521105,0.3153873,1.802971,0.4359790,0.8965622,IL1B..CD14.monocyte.PB00001.01_Female_BR1
IL1B..CD14.monocyte.PB00002.01_Male_BR1,0.8267947,1.499722,0.1896476,0.3446376,1.20068510,0.2702370,0.8664402,0.6309584,0.0000000,0.3246152,⋯,1.0598155,0.0000000,1.0461003,0.3246152,0.0000000,0.3706055,2.202253,0.1771037,1.6173190,IL1B..CD14.monocyte.PB00002.01_Male_BR1
IL1B..CD14.monocyte.PB00003.01_Female_BR1,1.0074245,0.572624,0.0000000,0.1164149,0.00000000,0.5425574,0.6734207,0.7315364,0.1164149,0.9467017,⋯,1.4168231,0.2696620,0.7838191,0.0000000,0.0000000,0.1842882,1.958701,0.0000000,0.8423586,IL1B..CD14.monocyte.PB00003.01_Female_BR1
IL1B..CD14.monocyte.PB00004.01_Male_BR1,1.1302947,1.285162,0.2181170,1.1987074,0.55374549,0.8469841,0.9965455,0.7241851,0.2293325,1.0225037,⋯,1.6834447,0.2793685,0.4779797,0.1875182,0.1200838,0.4924537,1.592984,0.4773711,1.7701366,IL1B..CD14.monocyte.PB00004.01_Male_BR1
IL1B..CD14.monocyte.PB00006.01_Female_BR1,0.7791579,1.460808,0.5123747,0.3467720,0.06202377,0.4096307,1.5528879,0.6565566,0.3090922,0.7221460,⋯,0.8891441,0.2998147,0.9152629,0.9778832,0.1912439,0.5341154,1.697275,0.1670615,0.3257211,IL1B..CD14.monocyte.PB00006.01_Female_BR1
IL1B..CD14.monocyte.PB00010.02_Male_BR1,1.0064725,1.101971,0.2458200,1.2418607,0.68290521,0.4408312,1.4681288,1.0327155,0.2519023,0.5140556,⋯,0.8436514,1.1180331,1.6009474,0.7034707,0.6336593,0.5660796,1.570037,0.3828403,1.4431669,IL1B..CD14.monocyte.PB00010.02_Male_BR1


In [120]:
dim(merged_df_v2)
dim(df)

[1] 25 37

[1] 92 25

In [121]:
data <- inner_join(df,merged_df_v2,by=c('ID'))

In [122]:
unique(data$cohort.cohortGuid)

[1] "BR2"

In [123]:
#write_csv(data, file = "/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/DEG_Analysis/MS_Sex_DEG_analysis/Mean_gene_expression_IL1B+_CD14_mono/Unscaled_IL1B_CD14mon_Male_BR1_mean_gene_X/IL1B_CD14mon_Male_BR1_mean_gene_X_v3.csv")

In [124]:
#write_csv(data, file = "/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/DEG_Analysis/MS_Sex_DEG_analysis/Mean_gene_expression_IL1B+_CD14_mono/Unscaled_IL1B_CD14mon_Male_BR1_mean_gene_X/IL1B_CD14mon_Male_BR2_mean_gene_X_v3.csv")

In [125]:
#write_csv(data, file = "/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/DEG_Analysis/MS_Sex_DEG_analysis/Mean_gene_expression_IL1B+_CD14_mono/Unscaled_IL1B_CD14mon_Male_BR1_mean_gene_X/IL1B_CD14mon_Female_BR1_mean_gene_X_v3.csv")

In [126]:
#write_csv(data, file = "/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/DEG_Analysis/MS_Sex_DEG_analysis/Mean_gene_expression_IL1B+_CD14_mono/Unscaled_IL1B_CD14mon_Male_BR1_mean_gene_X/IL1B_CD14mon_Female_BR2_mean_gene_X_v3.csv")